# Traffic Vision: Multi-Camera Recording System

## Objective
Develop a Linux system-level service for continuous multi-camera GDOT traffic stream recording.

## Technical Approach
- **System Level:** Bash scripts + systemd service for 24/7 operation
- **Recording Method:** Streamlink + FFmpeg (Method 3 from validation tests)
- **Data Source:** CSV file with camera definitions
- **Storage Structure:** `recordings/CAMERA-ID/YYYY-MM-DD/CAMERA-ID_YYYYMMDD_HHMMSS.mp4`

## Recording Configuration
- **Development/Testing:** 15-second segments (for workflow demo, included in repo)
- **Production Deployment:** 15-minute segments, 24/7 continuous on bare metal storage

## Current Implementation
- **Base Script:** `multi_camera_recorder.sh` (reads CSV, spawns parallel processes)
- **Target:** 8 Atlanta midtown cameras simultaneous recording

## Next Steps
1. Validate CSV data loading and stream accessibility
2. Test parallel recording with 2 cameras
3. Scale to full 8-camera deployment
4. Configure systemd service for continuous operation

**Date:** June 8, 2025  
**Branch:** `experiment/multi-camera-recording`